In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, roc_curve, precision_recall_curve,
    classification_report
)

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8,5)

# Load provider-level dataset (same file used in Notebook 02)
df = pd.read_csv("data/Provider_Level_Features.csv")
df.head()


In [ ]:
# Target selection
if "PotentialFraud_Binary" in df.columns:
    y = df["PotentialFraud_Binary"]
else:
    y = df["PotentialFraud"].map({"No": 0, "Yes": 1})

# Remove leakage columns
leak_cols = ["Provider", "PotentialFraud", "FraudLabel", "PotentialFraud_Binary"]
X = df.drop(columns=[c for c in leak_cols if c in df.columns])
